In [1]:
import pandas as pd
import datetime as dt
import os
import cisco_fiscal_calendar as cfc

In [2]:
today = dt.datetime.today().date()

In [3]:
today

datetime.date(2022, 4, 14)

In [4]:
USE_DATA_TYPES = {'Request ID':'Int64'}

## Read in all of the Regional Booking Files

In [5]:
Americas_path = r'C:\Users\phsheari\Documents\FIN BI Data\1_AMERICAS'
APJC_path = r'C:\Users\phsheari\Documents\FIN BI Data\2_APJC'
EMEAR_path = r'C:\Users\phsheari\Documents\FIN BI Data\3_EMEAR'

In [6]:
path_list = []
path_list.append(Americas_path)
path_list.append(APJC_path)
path_list.append(EMEAR_path)

In [7]:
path_list

['C:\\Users\\phsheari\\Documents\\FIN BI Data\\1_AMERICAS',
 'C:\\Users\\phsheari\\Documents\\FIN BI Data\\2_APJC',
 'C:\\Users\\phsheari\\Documents\\FIN BI Data\\3_EMEAR']

In [8]:
BookedFiles = []

for path in path_list:
    entries = os.listdir(path)
    for f_name in entries:
        if f_name.startswith('FINBI_') and f_name.endswith('.xlsx'):
            tempdf = pd.read_excel(path + '\\' + f_name, sheet_name='Compass Claim', dtype=USE_DATA_TYPES)
            tempdf['Respondant_File'] = f_name
            tempdf['AddIncre'] = today
            BookedFiles.append(tempdf)

C:\Users\phsheari\.conda\envs\compass_env\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [9]:
Booked = pd.concat(BookedFiles)

In [10]:
Booked['Request ID'] = Booked['Request ID'].astype(str)

In [11]:
Booked.rename(columns={'Date Readout Done':'Readout_Date', 'Compass Campaign Name':'Campaign'}, inplace=True)

In [12]:
Booked['CONCIERGE_EXTERNAL_READOUT_DT'] = pd.NA

In [13]:
Booked.columns

Index(['Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Sales Level 4',
       'CAV ID', 'CAV Name', 'GUID', 'GUID Name', 'FINBI Partner',
       'FINBI Deal ID', 'CX Product Category', 'Fiscal Quarter ID',
       'Fiscal Week ID', 'ReportWeek', 'Product Type', 'TCV', 'ACV', 'Ind',
       'DupeCheck', 'Request ID', 'Compass GUID', 'Compass GUID Name',
       'Compass Deal ID', 'Campaign', 'Threshold Date', 'Readout_Date',
       'Compass Advisor', 'Readout Delivery Status', 'Items Recommended',
       'Campaign Type', 'Service Source', 'CONCIERGE_ADVISOR_ASSIGNED',
       'CONCIERGE_INTERNAL_READOUT_DT', 'Target Campaign Flag',
       'Compass Exists?', 'Compass Claim?', 'Notes', 'Respondant_File',
       'AddIncre', 'CONCIERGE_EXTERNAL_READOUT_DT'],
      dtype='object')

In [14]:
def mark_partner_deals(compass_deal_id):
    if compass_deal_id == -44444 or compass_deal_id == 44444:
        return True
    else:
        return False

In [15]:
Booked['Partner Deal'] = Booked.apply(lambda x: mark_partner_deals(x['Compass Deal ID']), axis=1)

In [16]:
Booked.loc[Booked['Partner Deal']==True]

,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,CAV ID,CAV Name,GUID,GUID Name,FINBI Partner,FINBI Deal ID,...,CONCIERGE_ADVISOR_ASSIGNED,CONCIERGE_INTERNAL_READOUT_DT,Target Campaign Flag,Compass Exists?,Compass Claim?,Notes,Respondant_File,AddIncre,CONCIERGE_EXTERNAL_READOUT_DT,Partner Deal
43,EMEAR-REGION,EMEAR_MEA,COUNTRY_KSA,PUBLIC_SECTOR_KSA,2099103,NATIONAL INFORMATION CENTER SAUDI ARABIA,40357990,GOVERNMENT OF SAUDI ARABIA,NTT LTD COMMERCE CENTRE,55722259,...,NaN,NaT,False,Y,Y,NaN,FINBI_EMEAR_Bookings_FY22Q3_W9_W10.xlsx,2022-04-14,<NA>,True
44,EMEAR-REGION,EMEAR_OTHER,EMEAR_REBATES,EMEAR_CSPP,2099103,NATIONAL INFORMATION CENTER SAUDI ARABIA,40357990,GOVERNMENT OF SAUDI ARABIA,UNKNOWN,53411021,...,NaN,NaT,False,Y,Y,NaN,FINBI_EMEAR_Bookings_FY22Q3_W9_W10.xlsx,2022-04-14,<NA>,True


## Read In the FINBI Booking Accrual file

In [17]:
finbi_accrual_path = r'C:\Users\phsheari\Documents\FIN BI Data\FINBI_Bookings_Accrued_Results.xlsx'

In [18]:
Accrued = pd.read_excel(finbi_accrual_path, sheet_name='YTD Transactions', dtype=USE_DATA_TYPES)

In [19]:
Accrued['Request ID'] = Accrued['Request ID'].astype(str)

In [20]:
Accrued.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6312 entries, 0 to 6311
Data columns (total 42 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Sales Level 1                  6312 non-null   object        
 1   Sales Level 2                  6312 non-null   object        
 2   Sales Level 3                  6312 non-null   object        
 3   Sales Level 4                  6307 non-null   object        
 4   CAV ID                         6194 non-null   float64       
 5   CAV Name                       6207 non-null   object        
 6   GUID                           6312 non-null   int64         
 7   GUID Name                      6312 non-null   object        
 8   Partner                        5473 non-null   object        
 9   Booked Deal ID                 6312 non-null   int64         
 10  CX Product Category            6312 non-null   object        
 11  Fiscal Year      

In [21]:
Accrued['Request ID']

0          985
1          985
2          985
3          985
4          985
         ...  
6307     20355
6308     20355
6309    108570
6310    108570
6311    108570
Name: Request ID, Length: 6312, dtype: object

In [22]:
Booked.rename(columns={'Compass Campaign Name':'Campaign',
                       'FINBI Deal ID':'Booked Deal ID',
                       'FINBI Partner':'Partner',
                       'Product Type':'ProductType',
                       }, inplace=True)

In [23]:
dropcolumns = ['DupeCheck','IndexID','Threshold Date','Replacement','Old Request IDs','Unnamed: 43','MATCH','Concierge']

In [24]:
# Booked = Booked.drop(columns=dropcolumns)
for col in Booked.columns:
    if col in dropcolumns:
        Booked.drop(columns=col, inplace=True)

In [25]:
Booked.columns

Index(['Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Sales Level 4',
       'CAV ID', 'CAV Name', 'GUID', 'GUID Name', 'Partner', 'Booked Deal ID',
       'CX Product Category', 'Fiscal Quarter ID', 'Fiscal Week ID',
       'ReportWeek', 'ProductType', 'TCV', 'ACV', 'Ind', 'Request ID',
       'Compass GUID', 'Compass GUID Name', 'Compass Deal ID', 'Campaign',
       'Readout_Date', 'Compass Advisor', 'Readout Delivery Status',
       'Items Recommended', 'Campaign Type', 'Service Source',
       'CONCIERGE_ADVISOR_ASSIGNED', 'CONCIERGE_INTERNAL_READOUT_DT',
       'Target Campaign Flag', 'Compass Exists?', 'Compass Claim?', 'Notes',
       'Respondant_File', 'AddIncre', 'CONCIERGE_EXTERNAL_READOUT_DT',
       'Partner Deal'],
      dtype='object')

In [26]:
Booked = Booked[['Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Sales Level 4', 'CAV ID', 'CAV Name',
       'GUID', 'GUID Name', 'Partner', 'Booked Deal ID',
       'CX Product Category', 'Fiscal Quarter ID', 'Fiscal Week ID',
       'ReportWeek', 'ProductType', 'TCV', 'ACV', 'Ind', 'Request ID',
       'Readout_Date', 'Campaign Type', 'Compass Exists?',
       'Compass Claim?', 'Notes', 'Respondant_File', 'AddIncre', 'Campaign',
       'Target Campaign Flag', 'Partner Deal', 'Service Source',
       'Compass GUID Name', 'Compass GUID', 'Compass Deal ID', 'Compass Advisor',
       'Readout Delivery Status', 'Items Recommended','CONCIERGE_ADVISOR_ASSIGNED','CONCIERGE_INTERNAL_READOUT_DT','CONCIERGE_EXTERNAL_READOUT_DT']]

In [27]:
Booked.shape

(397, 39)

In [28]:
cal = cfc.get_calendar_dataframe()

In [29]:
Booked[['Sales Level 1', 'Sales Level 2', 'CAV ID', 'CAV Name',
       'GUID', 'GUID Name', 'Booked Deal ID',
       'CX Product Category', 'Fiscal Quarter ID', 'Fiscal Week ID',
       'ReportWeek', 'ProductType', 'TCV', 'Request ID',
       'Readout_Date', 'Campaign Type', 'Compass Exists?',
       'Compass Claim?',]]

,Sales Level 1,Sales Level 2,CAV ID,CAV Name,GUID,GUID Name,Booked Deal ID,CX Product Category,Fiscal Quarter ID,Fiscal Week ID,ReportWeek,ProductType,TCV,Request ID,Readout_Date,Campaign Type,Compass Exists?,Compass Claim?
0,Americas,US COMMERCIAL,479713,ACI WORLDWIDE INC,187793,ACI WORLDWIDE INC,54351297,SNTC,2022Q3,2022092,10,SNTC,48916.80,107613,2021-12-16,FIELD,Y,Y
1,Americas,GLOBAL ENTERPRISE SEGMENT,103157,ADVENTHEALTH,34694,ADVENTIST HEALTH SYSTEM SUNBELT INC,20188711,SNTC,2022Q3,2022091,9,SNTC,23581.64,102231,2021-11-18,TARGETING,Y,Y
2,Americas,GLOBAL ENTERPRISE SEGMENT,103157,ADVENTHEALTH,34694,ADVENTIST HEALTH SYSTEM SUNBELT INC,20188711,SNTC,2022Q3,2022092,10,SNTC,34544.21,102231,2021-11-18,TARGETING,Y,Y
3,Americas,US COMMERCIAL,103034,ADVENTIST HEALTH WEST,36063,ADVENTIST HEALTH SYSTEMWEST,54118622,SNTC,2022Q3,2022091,9,SNTC,27160.56,104922,2021-11-18,FIELD,Y,Y
4,Americas,AMERICAS_SP,107836,AMERICA MOVIL SAB DE CV,6547768,AMERICA MOVIL SAB DE CV,52595787,Expert Care,2022Q3,2022092,10,HTEC,3322620.00,20463,2021-07-09,FIELD,Y,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,EMEAR-REGION,EMEAR-CENTRAL,137629,VOESTALPINE AG,2367977,VOESTALPINE AG,55423540,Solution Support,2022Q3,2022082,6,SSPT,74785.51,101450,2022-01-21,TARGETING,Y,Y
114,EMEAR-REGION,EMEAR-CENTRAL,137629,VOESTALPINE AG,2367977,VOESTALPINE AG,52213775,Solution Support,2022Q3,2022084,8,SSPT,11009.38,101450,2022-01-21,TARGETING,Y,Y
115,EMEAR-REGION,EMEAR-UKI,669255,WPP PLC,201248059,WPP PLC,53611681,Solution Support,2022Q3,2022073,3,SSPT,18018.96,105757,2022-01-07,FIELD,Y,Y
116,EMEAR-REGION,EMEAR-CENTRAL,103144,ZURICH INSURANCE GROUP AG,4892203,ZURICH INSURANCE GROUP AG,44302318,Solution Support,2022Q3,2022074,4,SSPT,32496.99,101609,2022-01-19,TARGETING,Y,Y


In [30]:
weeks = cal.groupby(['finbi_fiscal_week_id',])[['week_of_fiscalyear']].min()

In [31]:
weeks.reset_index(inplace=True)
weeks.rename(columns={'finbi_fiscal_week_id':'Fiscal Week ID','week_of_fiscalyear':'Fiscal Week of Year'}, inplace=True)

In [32]:
Booked = Booked.merge(weeks, how='left')

In [33]:
Booked['Fiscal Week ID'] = Booked['Fiscal Week ID'].astype(str)
Booked['Fiscal Week of Year'] = Booked['Fiscal Week of Year'].astype(str)
Booked['Fiscal Year'] = Booked['Fiscal Quarter ID'].str.slice(stop=4)

In [34]:
Booked = Booked[['Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Sales Level 4', 'CAV ID', 'CAV Name',
       'GUID', 'GUID Name', 'Partner', 'Booked Deal ID', 'CX Product Category', 'Fiscal Year',
       'Fiscal Quarter ID', 'Fiscal Week ID', 'Fiscal Week of Year', 'ReportWeek', 'ProductType',
       'TCV', 'ACV', 'Ind', 'Request ID', 'Readout_Date', 'Campaign Type',
       'Compass Exists?', 'Compass Claim?', 'Notes', 'Respondant_File',
       'AddIncre', 'Campaign', 'Target Campaign Flag', 'Partner Deal',
       'Service Source', 'Compass GUID Name', 'Compass GUID', 'Compass Deal ID',
       'Compass Advisor', 'Readout Delivery Status', 'Items Recommended', 'CONCIERGE_ADVISOR_ASSIGNED','CONCIERGE_INTERNAL_READOUT_DT','CONCIERGE_EXTERNAL_READOUT_DT']]

In [35]:
Booked[['Fiscal Week of Year','Request ID']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397 entries, 0 to 396
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Fiscal Week of Year  397 non-null    object
 1   Request ID           397 non-null    object
dtypes: object(2)
memory usage: 9.3+ KB


In [36]:
# Booked['Fiscal Year']

In [37]:
Booked.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397 entries, 0 to 396
Data columns (total 41 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Sales Level 1                  397 non-null    object        
 1   Sales Level 2                  397 non-null    object        
 2   Sales Level 3                  397 non-null    object        
 3   Sales Level 4                  397 non-null    object        
 4   CAV ID                         397 non-null    int64         
 5   CAV Name                       397 non-null    object        
 6   GUID                           397 non-null    int64         
 7   GUID Name                      397 non-null    object        
 8   Partner                        397 non-null    object        
 9   Booked Deal ID                 397 non-null    int64         
 10  CX Product Category            397 non-null    object        
 11  Fiscal Year        

In [38]:
Accrued = Accrued.merge(weeks, how='left')

In [39]:
NewAccrued = pd.concat([Accrued,Booked])

In [40]:
# NewAccrued['Request ID'] = NewAccrued['Request ID'].astype(str)

In [41]:
NewAccrued['Fiscal Week ID'] = NewAccrued['Fiscal Week ID'].astype(str)
NewAccrued['Fiscal Week of Year'] = NewAccrued['Fiscal Week of Year'].astype(str)
NewAccrued['Fiscal Year'] = NewAccrued['Fiscal Quarter ID'].str.slice(stop=4)

In [42]:
NewAccrued = NewAccrued[['Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Sales Level 4', 'CAV ID', 'CAV Name',
       'GUID', 'GUID Name', 'Partner', 'Booked Deal ID', 'CX Product Category', 'Fiscal Year',
       'Fiscal Quarter ID', 'Fiscal Week ID', 'Fiscal Week of Year', 'ReportWeek', 'ProductType',
       'TCV', 'ACV', 'Ind', 'Request ID', 'Readout_Date', 'Campaign Type',
       'Compass Exists?', 'Compass Claim?', 'Notes', 'Respondant_File',
       'AddIncre', 'Campaign', 'Target Campaign Flag', 'Partner Deal',
       'Service Source', 'Compass GUID Name', 'Compass GUID', 'Compass Deal ID',
       'Compass Advisor', 'Readout Delivery Status', 'Items Recommended','CONCIERGE_ADVISOR_ASSIGNED','CONCIERGE_INTERNAL_READOUT_DT','CONCIERGE_EXTERNAL_READOUT_DT']]

In [43]:
NewAccrued['Request ID']

0         985
1         985
2         985
3         985
4         985
        ...  
392    101450
393    101450
394    105757
395    101609
396    101609
Name: Request ID, Length: 6709, dtype: object

In [44]:
NewAccrued['Fiscal Year'].unique()

array(['2020', '2021', '2022'], dtype=object)

In [45]:
NewAccrued.loc[(NewAccrued['Fiscal Quarter ID'].str.startswith('2022') & (NewAccrued['Fiscal Quarter ID'].str.endswith('Q2'))) & (NewAccrued['Sales Level 1']=='Americas')].TCV.sum()

172404333.01417854

In [46]:
NewAccrued.to_excel(r'C:\Users\phsheari\Documents\FIN BI Data\FINBI_Bookings_Accrued_Results - Copy2NewAccrued.xlsx', engine='xlsxwriter', sheet_name='YTD Transaction', index=False)

In [47]:
Booked.to_excel(r'C:\Users\phsheari\Documents\FIN BI Data\FINBI_INCREMENTAL_Booked.xlsx', engine='xlsxwriter', sheet_name='2Week Transactions', index=False)